In [1]:
from destination_generator import DestinationGenerator

d:\anaconda\envs\CityNav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using custom box path: d:\github\simworld\clpg\clpg\..\data\Bbox.json


In [2]:
destination_generator = DestinationGenerator()

In [3]:
import os
city_path = os.path.join(os.getcwd(), "input_map", "total", "map_roud_10_0", "progen_world_with_road_name.json")
road_path = os.path.join(os.getcwd(), "input_map", "total", "map_roud_10_0", "roads.json")
destination_generator.load_env(city_path, road_path, mode='buildings', if_load_map=True)

Importing city data from d:\github\SimWorld\task_generator\input_map\total\map_roud_10_0\progen_world_with_road_name.json
Import completed successfully
Imported road segments: 10
Imported buildings: 91
Imported details: 549


INFO:__init__:230:Got connection confirm: b'connected to gym_citynav'


=>Info: using ip-port socket


In [5]:
output_path = os.path.join(os.getcwd(), "output", "multi_agent", "map_roud_10_0" )
distance_units_to_building = [15, 15]
img, static_description, surrounding_context, description = destination_generator.generate_task(output_path, distance_units_to_building)

building_node: Node(position=Vector(x=6680.0, y=1700.0), type=normal)
spawning_locations: [Vector(x=21020.0, y=1700.0), Vector(x=5020.0, y=-1700.0)]
spawning_orientations: [270, 90]
image_path_name: ['destination.png']
description: Look for a tall beige building with large arched glass windows at the base. It's right next to another similar building, and you'll see a tree nearby on the right side. Keep an eye out for the uniform rectangular windows on the upper floors.
task_config: {'criteria': {'max_steps': 150, 'dist_threshold': 200}, 'agents': [{'type': 'spot', 'instance_name': 'BP_SpotRobot_C', 'agent_name': 'Agent_1', 'radius': 70, 'benchmark_type': 1, 'camera_id': 1, 'destination': {'x': 6881.399600000001, 'y': 1700.0, 'z': 73.957685}, 'spawning_point': {'location': {'x': 21020.0, 'y': 1700.0, 'z': 73.957685}, 'orientation': {'roll': 0, 'pitch': 0, 'yaw': 270}}, 'instruction': "Look for a tall beige building with large arched glass windows at the base. It's right next to another 

In [5]:
destination_generator.exit()

In [26]:
front_door_position = destination_generator.assets['GEN_BP_Building_43_C_10']['front_door_position']
front_door_position = [front_door_position[0]*100, (front_door_position[1]-20-17)*100]


In [ ]:
front_door_position

In [ ]:
front_door_position = destination_generator.assets['GEN_BP_Building_43_C_10']['front_door_position']

destination_generator.unrealcv_client.client.request("vset /camera/0/location {} {} {}".format(front_door_position[0]*100, (front_door_position[1]-40)*100, 200))

In [ ]:
distances = [(((n.position.x - front_door_position[0])**2 + (n.position.y - front_door_position[1])**2)**0.5, n) for n in destination_generator.map.nodes]
closest_nodes = sorted(distances, key=lambda x: x[0])[:3]

for _, node in closest_nodes:
    if node.type == "normal":
        destination_node = node
        break

distance = 10

distance_threshold = 1
normal_nodes = []
for i in range(distance_threshold+1):
    end_nodes = destination_generator.map.get_random_node_with_edge_distance([destination_node], min_distance=distance+i, max_distance=distance+i)
    for node in end_nodes:
        if node.type=="normal":
            normal_nodes.append((distance+i, node))
    end_nodes = destination_generator.map.get_random_node_with_edge_distance([destination_node], min_distance=distance-i, max_distance=distance-i)
    for node in end_nodes:
        if node.type=="normal":
            normal_nodes.append((distance-i, node))
    if len(normal_nodes) >= 2:
        break

    

In [ ]:
import random
random.choice(normal_nodes)[1].position
# normal_nodes[0][1].position

In [ ]:
for i in range(2):
    print(i)


In [ ]:
print(nodes)

In [ ]:
destination_generator.map.get_random_node()

In [ ]:
static_description

In [ ]:
surrounding_context

In [ ]:
asset_id = destination_generator.select_random_asset()
destination_generator.generate_template_description(asset_id)

In [ ]:
print(asset_id)

In [9]:
from llm_utils.customized_openai_model import CustomizedOpenAIModel
import numpy as np
import json

def _process_image_to_base64(image: np.ndarray) -> str:
    import cv2
    import io
    from PIL import Image
    import base64    
    """Convert numpy array image to base64 string.

    Args:
        image (np.ndarray): Image array (1 or 3 channels)

    Returns:
        str: Base64 encoded image string
    """
    # Convert single channel to 3 channels if needed
    if len(image.shape) == 2 or (len(image.shape) == 3 and image.shape[2] == 1):
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # Ensure uint8 type
    if image.dtype != np.uint8:
        image = (image * 255).astype(np.uint8)

    # Convert to PIL Image
    pil_image = Image.fromarray(image)

    # Convert to base64
    buffered = io.BytesIO()
    pil_image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    return f"data:image/jpeg;base64,{img_str}"

In [ ]:
model = CustomizedOpenAIModel("gpt-4o")

response = model.generate(
    [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that writes short, natural directions to help someone find an urban building. "
                "You will receive a front‑facing street‑level photo of the building and a structured summary of its features and nearby context.\n\n"
                "Your job is to produce 2–3 friendly sentences that guide a person (or low‑height robot) to spot this building in real life. "
                "Mention 1–2 of the building’s most distinctive features at eye‑level and reference one clear, nearby visual clue—like a sign, bench, lamp post, or neighboring building. "
                "Avoid technical terms, measurements, or high‑up details like rooftops—focus on what’s visible from about 70 cm off the ground."
            )
        },
        {
            "role": "user",
            "content": [
                { "type": "text", "text": "Here is a street‑facing image of the building." },
                { "type": "image_url", "image_url": { "url": _process_image_to_base64(np.array(img)) } },
                { "type": "text", "text": (
                    "Structured building info:\n" + json.dumps(static_description, indent=2)
                    + "\n\nNearby context:\n" + json.dumps(surrounding_context, indent=2)
                ) }
            ]
        }
    ]
)
response[0][0]


In [ ]:
for asset_id, asset_info in destination_generator.assets.items():
    print(asset_info)
    print(asset_id)
    break

In [ ]:
destination_generator.roads

In [ ]:
from llm_utils.customized_openai_model import CustomizedOpenAIModel

temp_template = destination_generator.generate_template_description()
while temp_template["exact_matches"] != 1:
    temp_template = destination_generator.generate_template_description()
desc = temp_template

print("================INPUT TEMPLATE================")
print(desc)

model = CustomizedOpenAIModel("gpt-4o")
response = model.generate(
    [
        {"role": "system", "content":  "You are a helpful assistant that writes short, natural descriptions of urban buildings. "
                "Each description should help someone easily spot the building in real life by mentioning 1–2 striking visual features and only the most relevant surrounding clue. "
                "Keep it concise, conversational, and free of technical details or exact numbers. Think like you're giving someone directions in person."
            },
        {"role": "user", "content": f"""Describe the building below in 2–3 short sentences. Focus on its most eye-catching feature and the clearest nearby landmark.
         static_description: {desc['static_description']}
         surrounding_context: {desc['surrounding_context']}"""}
    ]
)
print("================OUTPUT DESCRIPTION================")
print(response[0][0])
